# Migrate or clone a lakeFS repository on Azure

#### Use this notebook if you want to migrate/clone a source repository to a target repository within the same lakeFS environment or in different lakeFS environments

## Prerequisites

#### 1. Source and a target lakeFS environments (you can [deploy one independently](https://docs.lakefs.io/deploy/) or use the hosted solution [lakeFS Cloud](https://lakefs.cloud))
#### 2. Source repository
#### 3. Storage Container for target repository but don't create target repository ahead of time (you will create a target repo in this notebook)

## Setup Task: Import required Python packages

In [ ]:
%xmode Minimal
import lakefs_client
from lakefs_client import models
from lakefs_client.client import LakeFSClient
import random
import os
import datetime

## Setup Task: Change your lakeFS credentials for the source lakeFS environment

In [ ]:
sourceLakefsEndPoint = '<Source lakeFS Endpoint URL>' # e.g. 'https://username.aws_region_name.lakefscloud.io'
sourceLakefsAccessKey = '<Source lakeFS Access Key>'
sourceLakefsSecretKey = '<Source lakeFS Secret Key>'

## Setup Task: Change your lakeFS credentials for the target lakeFS environment

#### If your source and target environments are same then use same credentials for target as you specified for the source above

In [ ]:
targetLakefsEndPoint = '<Target lakeFS Endpoint URL>' # e.g. 'https://username.aws_region_name.lakefscloud.io'
targetLakefsAccessKey = '<Target lakeFS Access Key>'
targetLakefsSecretKey = '<Target lakeFS Secret Key>'

## Setup Task: Change lakeFS repo names

In [ ]:
source_repo_name = "source-repo"
target_repo_name = "target-repo"

## Setup Task: Change main/production branch name for the source repo

In [ ]:
source_main_branch = "main"

## Setup Task: Change storage account names and container names for the source & target

#### Storage account name can be same for the source and target

In [ ]:
source_storage_namespace = 'https://source-storage-account-name.blob.core.windows.net/sourceContainer'
target_storage_namespace = 'https://target-storage-account-name.blob.core.windows.net/targetContainer'

## Setup Task: Change SAS Tokens

#### You will copy data from source Storage Container to the target Storage Container by using the [azcopy](https://learn.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10?toc=%2Fazure%2Fstorage%2Fblobs%2Ftoc.json&bc=%2Fazure%2Fstorage%2Fblobs%2Fbreadcrumb%2Ftoc.json) software (which is pre-installed in this container) and you will use Shared Access Signatures (SAS) token to [Authorize azcopy](https://learn.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10?toc=%2Fazure%2Fstorage%2Fblobs%2Ftoc.json&bc=%2Fazure%2Fstorage%2Fblobs%2Fbreadcrumb%2Ftoc.json#authorize-azcopy).

#### Use this document to [Create SAS tokens for your storage containers](https://learn.microsoft.com/en-us/azure/cognitive-services/translator/document-translation/how-to-guides/create-sas-tokens?tabs=Containers). Select "Read" and "List" permissions for Source Container and "Write" permission for Target Container while creating SAS Tokens.

In [ ]:
source_container_SAS_token = 'source_container_SAS_token'

In [ ]:
target_container_SAS_token = 'target_container_SAS_token'

## Setup Task: Create lakeFS Python client for source lakeFS environment

In [ ]:
if not 'sourceClient' in locals():
    # lakeFS credentials and endpoint
    configuration = lakefs_client.Configuration()
    configuration.username = sourceLakefsAccessKey
    configuration.password = sourceLakefsSecretKey
    configuration.host = sourceLakefsEndPoint

    sourceClient = LakeFSClient(configuration)
    print("Created source lakeFS client.")

## Setup Task: Create lakeFS Python client for target lakeFS environment

In [ ]:
if not 'targetClient' in locals():
    # lakeFS credentials and endpoint
    configuration = lakefs_client.Configuration()
    configuration.username = targetLakefsAccessKey
    configuration.password = targetLakefsSecretKey
    configuration.host = targetLakefsEndPoint

    targetClient = LakeFSClient(configuration)
    print("Created target lakeFS client.")

# Step 1 - Commit Changes

## IMPORTANT: Uncommitted data is not migrated over so check uncommitted data (this might take time if you have many branches in the source repo)

In [ ]:
has_more = True
after = ""

while has_more:
    list_branches = sourceClient.branches.list_branches(
        repository=source_repo_name,
        after=after)

    for branch in list_branches.results:
        get_diff = sourceClient.branches.diff_branch(
            repository=source_repo_name,
            branch=branch.id,
            amount=1)
        if get_diff.results:
            print('Branch with uncommitted data: ' + branch.id)

    # pagination
    has_more = list_branches.pagination.has_more
    after = list_branches.pagination.next_offset

## OPTIONAL: Commit any uncommitted data in your source repo (this might take time if you have many branches in the source repo)
#### If you want, you can manually commit the changes for the branches listed above from the lakeFS UI

#### You can run previous command again after finishing this to verify that there are not any uncommitted data any more

In [ ]:
has_more = True
after = ""

while has_more:
    list_branches = sourceClient.branches.list_branches(
        repository=source_repo_name,
        after=after)

    for branch in list_branches.results:
        get_diff = sourceClient.branches.diff_branch(
            repository=source_repo_name,
            branch=branch.id,
            amount=1)
        if get_diff.results:
            print('Committed changes for Branch: ' + branch.id)
            sourceClient.commits.commit(
                repository=source_repo_name,
                branch=branch.id,
                commit_creation=models.CommitCreation(
                    message='Added a file!'))
    # pagination
    has_more = list_branches.pagination.has_more
    after = list_branches.pagination.next_offset

# Step 2 - Backup Metadata of Source Repository

In [ ]:
dump_refs = sourceClient.refs.dump_refs(
    repository=source_repo_name)

print(dump_refs)

# Step 3 - Copy Data from Source to Target

In [ ]:
azureCopyCommand = "azcopy copy '" + source_storage_namespace + "?" + source_container_SAS_token + "' '" + target_storage_namespace + "?" + target_container_SAS_token + "' --recursive"
print(azureCopyCommand)

! $azureCopyCommand

## Step 4 - Create Target Bare Repository

In [ ]:
targetClient.repositories.create_repository(
    repository_creation=models.RepositoryCreation(
        name=target_repo_name,
        storage_namespace=target_storage_namespace,
        default_branch=source_main_branch),
    bare=True)

## Step 5 - Restore Metadata to Target Repository

In [ ]:
targetClient.refs.restore_refs(
    repository=target_repo_name,
    refs_dump=dump_refs)

print('Repository migrated')

## More Questions?

###### Join the lakeFS Slack group - https://lakefs.io/slack